# **Lab 2 - Explainable and Trustworthy AI**


---



**Eleonora Poeta** (eleonora.poeta@polito.it)

**Lab 2:** Global post-hoc explainable models on structured data

# **Permutation Feature Importance**


---



*	Permutation feature importance is a model inspection technique that measures the **contribution** of **each feature** to the **model's performances** on a  given tabular dataset.
* A **feature** is **important** if shuffling its values, the ***model error increases***. So, the model relied on that feature for the prediction.


Key **advantages** of Permutation Feature importance technique:


> * Nice and **direct interpretation** of the model's behaviour.
* It is **model-agnostic**.
* It does **not require retraining the model**.



Main **disadvantages** of Permutation Feature importance technique:


> * It assumes the **Feature independence**. If features are correlated, it can be biased by unrealistic data instances.
* It is strictly linked to the **model performance measures**. In some cases other measures can be of interest.




---


## **Exercise 1**

The [**Titanic**](https://www.openml.org/search?type=data&sort=runs&id=40945&status=active) dataset describes the survival status of individual passengers on the Titanic. In this exercise you have to:

* **Preprocess** the Titanic dataset. You can follow these main steps:
> * **Load** the dataset
  * **Split** the dataset into training and test set using the **80/20** ratio. **Shuffle** the dataset and **stratify** it using the target variable.
  * Fill **null** values. `age` column with the mean, `fare` with the median and `embarked` with the most frequent values.
  * Encoding
    * **Remove** columns that are *not informative for the final task*, or that *contain information about target variable*.
    * Perform **OneHotEncoding** and **OrdinalEncoder**, where needed.
    * Perform **MinMax** scaling, where needed,
    * We suggest to use the **[ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)** module

* Fit a **[RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)()** over the Titanic dataset. We suggest to use the **[Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)** module
* Inspect model's **accuracy** on training and test dataset.

### **Exercise 1.1**
* Calculate **Feature Importances** (as previously done in Lab 1):
> * What can you infer? What is the most important feature? Is it categorical or numerical?
> Note that you can aggregate (via sum) the feature importance for the feature value importance of the same categorical attribute to have the importance of the attribute

### **Exercise 1.2**
* Now, compute [**permutation_importance**](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html).
>   * **Firstly**, compute this for the **test set**. What are the most important features?
    * Re-do the computation of permutation_importance, now **on the training set**. Did you obtain the same result as for the test set? If not, what can be the problem?

### **Exercise 1.3**
* Let's now instanciate another **RandomForestClassifier** with parameter `min_samples_leaf=20`.
> * Repeat the steps for computing the permutation_importance on both training and test set.
  * Has something changed? What can you infer about RandomForestClassifier's behaviour with respect to overfitting?


***Hint***:

> In the first case, the results of permutation_importance on the training and test set can be explained by overfitting of the RandomForestClassifier. So, the RF has capacity to use the feature `fare` to overfit.












## **Solution:**

In [1]:
# If your dataset is stored on Google Drive, mount the drive before reading it
from google.colab import drive
drive.mount('/content/drive')

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import numpy as np

MessageError: Error: credential propagation was unsuccessful

### Data preprocessing - Titanic dataset

In [55]:
#Importing csv file
df = pd.read_csv('./titanic.csv')

#Splitting dataset
df_train, df_test = train_test_split(df, test_size = 0.2, train_size = 0.8, shuffle = True, random_state=42, stratify = df['survived'])

#Finding null-values and duplicates
# print(f'Null vals train set: ', df_train.isna().sum()) #Some null values are present, filling them accordingly to the lab text
# print(f'Null vals test set: ', df_test.isna().sum()) #Some null values are present, filling them accordingly to the lab text
# print(f"Number of duplicate rows train set: {df_train.duplicated(keep=False).sum()}") #No duplicates in the dataframe
# print(f"Number of duplicate rows test set: {df_test.duplicated(keep=False).sum()}") #No duplicates in the dataframe

In [56]:
#Filling null values
df_train['age'].fillna(df_train['age'].mean(), inplace = True)
df_test['age'].fillna(df_test['age'].mean(), inplace = True)

df_train['fare'].fillna(df_train['fare'].median(), inplace = True)
df_test['fare'].fillna(df_test['fare'].median(), inplace = True)

# Filling with most_frequent value: METHOD 1
df_train['embarked'].fillna(df_train['embarked'].mode()[0], inplace = True)
df_test['embarked'].fillna(df_test['embarked'].mode()[0], inplace = True)

# Filling with most_frequent value: METHOD 2
# imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
# df_train[['embarked']] = imp.fit_transform(df_train[['embarked']])
# df_test[['embarked']] = imp.transform(df_test[['embarked']])

# print(f'Null vals train set: ', df_train.isna().sum())
# print(f'Null vals test set: ', df_test.isna().sum())

In [57]:
#Feature selection
df_train.drop(columns = ['name', 'ticket', 'cabin', 'boat', 'body', 'home.dest'], inplace = True)
df_test.drop(columns = ['name', 'ticket', 'cabin', 'boat', 'body', 'home.dest'], inplace = True)
#If dataset same as Lab0, no correlation between features, so no other columns to drop

In [58]:
# FEATURE ENGINEERING

# Discretization
age_category = ['Child (0-14]', 'Young (14-24]', 'Adults (24-50]', 'Senior (50+]']

df_train['age_disc'] = pd.cut(df_train['age'], bins = [0, 14, 24, 50, 150], labels = age_category)
df_train.drop(columns = ['age'], inplace = True)
df_test['age_disc'] = pd.cut(df_test['age'], bins = [0, 14, 24, 50, 150], labels = age_category)
df_test.drop(columns = ['age'], inplace = True)

# Encoding
categorical_columns = ['sex', 'embarked']
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(df_train[categorical_columns])
#Train
tmp_df_train = pd.DataFrame(data = ohe.transform(df_train[categorical_columns]).toarray(), columns = ohe.get_feature_names_out())
df_train_encoded = df_train.copy()
df_train_encoded.drop(columns = categorical_columns, axis = 1, inplace = True)
df_train_encoded = pd.concat([df_train_encoded.reset_index(drop = True), tmp_df_train], axis = 1)
#Test
tmp_df_test = pd.DataFrame(data = ohe.transform(df_test[categorical_columns]).toarray(), columns = ohe.get_feature_names_out())
df_test_encoded = df_test.copy()
df_test_encoded.drop(columns = categorical_columns, axis = 1, inplace = True)
df_test_encoded = pd.concat([df_test_encoded.reset_index(drop = True), tmp_df_test], axis = 1)

#TO DO: Encodeother features

In [59]:
df_train.head()

,pclass,sex,sibsp,parch,fare,embarked,survived,age_disc
999,3,female,0,0,7.7500,Q,1,Adults (24-50]
392,2,female,1,0,27.7208,C,1,Young (14-24]
628,3,female,4,2,31.2750,S,0,Child (0-14]
1165,3,male,0,0,7.2250,C,0,Adults (24-50]
604,3,female,0,0,7.6500,S,1,Young (14-24]


In [60]:
# Splitting data and labels
y_train = df_train_encoded['survived']
x_train = df_train_encoded.drop('survived', axis = 1)
y_test = df_test_encoded['survived']
x_test = df_test_encoded.drop('survived', axis = 1)

#### Fit the Random Forest Classifier

In [61]:
rf_clf = RandomForestClassifier()
rf_clf.fit()

TypeError: BaseForest.fit() missing 2 required positional arguments: 'X' and 'y'

#### Model's performances

In [ ]:
### Write your solution here

### Exercise 1.b

#### Calculate the Feature Importances

In [ ]:
### Write your solution here

### Exercise 1.b

#### Permutation Importances

In [ ]:
### Write your solution here

### Exercise 1.3

Set the **min_samples_leaf at 20** and fit a random forest classifier

In [ ]:
### Write your solution here

New Model's accuracy score

In [ ]:
### Write your solution here

Permutation importances

In [ ]:
### Write your solution here

# **Partial dependence plot**


---



The **Partial Dependence Plot** (in short, PDP) shows the **marginal effect** of one or two features **on the predicted outcome** of a Machine learning model.

* It is a **global method**: The method considers **all instances** and gives a statement about the **global relationship** of a feature with the predicted outcome.

Key **Advantages**:

* The PDP computation is really **intuitive**.
* By providing an **explanation** in the form of a **visualization**, it is easy to inspect.


Main **Disadvantages**:

* The **assumption** of **independence** is the biggest issue with PDP plots.
* The **realistic maximum number of features** in a partial dependence function is **two**.



---

# **Exercise 2**

* Instanciate and Fit a [**DecisionTreeClassifier**](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) and a [**RandomForestClassifier**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) over Titanic dataset.
* **Evaluate** both models by inspecting their **accuracy** scores over train and test dataset.
* Print **Feature Importances** for both models.
* Generate the **PDP** using [**PartialDependenceDisplay.from_estimator**](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.PartialDependenceDisplay.html) function for both models.


## **Solution:**

RF Classifier

In [ ]:
### Write your solution here

DT classifier

In [ ]:
### Write your solution here

Plot the feature importance for Random Forest

In [ ]:
### Write your solution here

Plot the feature importance for decision tree

In [ ]:
### Write your solution here

Generate PDP for Random Forest Classifier

In [ ]:
### Write your solution here

Generate PDP for Decision Tree Classifier

In [ ]:
### Write your solution here

# **Global surrogate models**


---



The purpose of (interpretable) surrogate models is to **approximate** the **predictions** of the underlying model *as accurately as possible* and to be interpretable at the same time.

The surrogate model is a **model-agnostic method**, since it does **not require** any **information** about the **inner workings of the black box model**, *only access to data and the prediction function is necessary*.

To **obtain a surrogate model** you have to perform the following steps:



1.   Select a dataset X.
2.   For the selected dataset X, get the predictions of the black box model.
3.   Select an interpretable model type (linear model, decision tree, …).
4.   Train the interpretable model on the dataset X and its predictions.

Congratulations! You now have a surrogate model!

Measure **how well** the **surrogate model replicates the predictions of the black box model** to interpret the model.



---

## **Exercise 3:**

The aim of this exercise is to use a white-box model, that is the surrogate model, to explain the black-box model. You have to:

* Instanciate and fit the **black-box** model, i.e. the [**GradientBoosterClassifier**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) over the Titanic
dataset.
> * You can retrieve the predictions with the *_predictions* function.



* Instanciate and fit the **white-box** model, i.e. the [**LogisticRegression**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) over the Titanic dataset **and** the predictions of the black-box model.
* Calculate the **accuracy_score** for both models on the training and test dataset.
* Calculate the **reconstruction_error** for both models on the training and test dataset.
* **Print** the obtained **results**.

***Hint:***
> You can inted the **reconstruction_error** as the loss of information or performance when using the predictions of the black-box model to train the white-box model.

## **Solution:**

Train the black-box model

In [ ]:
### Write your solution here

Train the white-box model over the Titanic dataset and the predictions from the black-box model.

In [ ]:
### Write your solution here

Inspect the coefficients

In [ ]:
### Write your solution here

Evaluate the performances of black-box and white-box models.

In [ ]:
### Write your solution here

Evaluate the reconstruction error

In [ ]:
### Write your solution here